In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio

import numpy as np
import tensorflow as tf

import joblib

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
root_path = '/content/drive/MyDrive/INF2475/2412841'
models_path = f"{root_path}/resources/models"

In [4]:
scaler = joblib.load(root_path+'/resources/scalers/scaler.gz')

In [5]:
eval_dict = {}
data_dict = {}
TIME_STEPS = [90, 90, 60, 60, 30, 30]

for i in range(1,7):

    model = tf.keras.models.load_model(models_path+f"/model{i}/model{i}.h5", compile = False)

    X_train = np.load(
        f'{root_path}/data/model{i}_X_train.npy'
    )

    y_train = np.load(
        f'{root_path}/data/model{i}_y_train.npy'
    )

    X_test = np.load(
        f'{root_path}/data/model{i}_X_test.npy'
    )

    y_test = np.load(
        f'{root_path}/data/model{i}_y_test.npy'
    )

    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)

    # Ajustando a forma de y_train e y_test para ser 2D
    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    # Invertendo a normalização dos predicts
    train_predict = scaler.inverse_transform(train_predict)
    test_predict = scaler.inverse_transform(test_predict)

    # Invertendo a normalização de y_train e y_test
    y_train = scaler.inverse_transform(y_train)
    y_test = scaler.inverse_transform(y_test)

    mae = round(mean_absolute_error(y_test, test_predict),2)
    mape = round(mean_absolute_percentage_error(y_test, test_predict)*100,2)

    eval_dict[f'model{i}'] = [mae, mape]
    data_dict[f'model{i}'] = {
        'train_predict': train_predict,
        'test_predict': test_predict,
        'X_train': X_train,
        'X_test': X_test,
        'time_steps': TIME_STEPS[i-1]
    }


19/19 [==============================] - 0s 5ms/step


In [6]:
eval_metrics = pd.DataFrame(eval_dict, index=['MAE', 'MAPE']).T
eval_metrics

,MAE,MAPE
model1,1779.60,2.33
model2,1478.09,1.99
model3,1570.52,2.10
model4,1441.23,1.95
model5,1653.74,2.21
model6,1797.96,2.40


In [7]:
# Model 1: time_steps = 90, batch_sizes = 64
# Model 2: time_steps = 90, batch_sizes = 32

# Model 3: time_steps = 60, batch_sizes = 64
# Model 4: time_steps = 60, batch_sizes = 32

# Model 5: time_steps = 30, batch_sizes = 64
# Model 6: time_steps = 30, batch_sizes = 32

In [8]:
eval_metrics

,MAE,MAPE
model1,1779.60,2.33
model2,1478.09,1.99
model3,1570.52,2.10
model4,1441.23,1.95
model5,1653.74,2.21
model6,1797.96,2.40


In [9]:
selected_model = eval_metrics['MAE'].idxmin()
selected_model

'model4'

In [10]:
df = pd.read_excel(root_path+'/data/dados_carga_sin.xlsx')
df.index = pd.to_datetime(df['data'])
df.pop('data')
df['carga'] = df['carga'].astype(float)

In [11]:
def save_models_plots(models_data_dict, df):

    figs = []

    fig = make_subplots(rows=len(models_data_dict), cols=1,
                        row_heights=[200]*len(models_data_dict))

    for i in range(1,len(models_data_dict)+1):

        curr_model = f'model{i}'

        time_step = models_data_dict[curr_model]['time_steps']

        X_train = models_data_dict[curr_model]['X_train']
        X_test = data_dict[curr_model]['X_test']

        train_predict = data_dict[curr_model]['train_predict']
        test_predict = data_dict[curr_model]['test_predict']

        train_size = len(X_train)
        test_size = len(X_test)

        df_plot = pd.DataFrame(index=df.index)
        df_plot['Valores reais'] = df['carga']
        df_plot['Predições no treino'] = np.nan
        df_plot['Predições no teste'] = np.nan
        df_plot.iloc[time_step:len(train_predict) + time_step, 1] = train_predict.reshape(-1)
        df_plot.iloc[train_size + time_step:train_size + time_step + test_size, 2] = test_predict.reshape(-1)

        # Plotting using Plotly Express
        plot = px.line(df_plot, x=df_plot.index, y=df_plot.columns,
                        title=f'(Model {i}) Predição de Carga de Energia',
                        labels={'index': 'Date', 'value': 'Carga'},
                        template='plotly_white', width=1200, height=500)

        traces = plot['data']
        for trace in traces:
            fig.add_trace(trace, row=i, col=1)

        plot.write_html(f'{root_path}/resources/plots/{curr_model}.html')

        if i == 4:
            plot.show()

    pio.write_html(fig, file=f'{root_path}/resources/plots/all_models.html', auto_open=True)


In [12]:
save_models_plots(data_dict, df)